<a href="https://colab.research.google.com/github/stefvas/neural-networks/blob/main/lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo pip install --upgrade pip
!sudo pip install --upgrade tensorflow

     |████████████████████████████████| 2.1 MB 15.8 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 KB 14.4 MB/s eta 0:00:00


In [30]:
from __future__ import absolute_import, division, print_function, unicode_literals # legacy compatibility

import tensorflow as tf
import tensorflow.keras.applications as app
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Activation, SpatialDropout2D, ZeroPadding2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras import layers,callbacks,regularizers


import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import time
import timeit
!pip install tf-explain
!pip install pyyaml h5py  # Required to save models in HDF5 format

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 4.6 MB/s eta 0:00:00


In [18]:
# Download image files
image_zip = tf.keras.utils.get_file('flickr30k-images-ecemod.zip',
                                      cache_subdir=os.path.abspath('.'),
                                      origin='https://spartacus.1337.cx/flickr-mod/flickr30k-images-ecemod.zip',
                                      extract=True)
os.remove(image_zip)

4376395776/4376381805 [==============================] - 211s 0us/step


In [19]:
# Download train captions file
train_captions_file = tf.keras.utils.get_file('train_captions.csv',
                                           cache_subdir=os.path.abspath('.'),
                                           origin='https://spartacus.1337.cx/flickr-mod/train_captions.csv',
                                           extract=False)

# Download test files list
test_images_file = tf.keras.utils.get_file('test_images.csv',
                                           cache_subdir=os.path.abspath('.'),
                                           origin='https://spartacus.1337.cx/flickr-mod/test_images.csv',
                                           extract=False)

In [26]:
!pwd
!ls


/content
gdrive	image_dir  sample_data	test_images.csv  train_captions.csv


In [31]:
# helper functions

# select from from_list elements with index in index_list
def select_from_list(from_list, index_list):
  filtered_list= [from_list[i] for i in index_list]
  return(filtered_list)

# append in filtered_list the index of each element of unfilterd_list if it exists in in target_list
def get_ds_index(unfiliterd_list, target_list):
  index = 0
  filtered_list=[]
  for i_ in unfiliterd_list:
    if i_[0] in target_list:
      filtered_list.append(index)
    index += 1
  return(filtered_list)

# select a url for a unique subset of CIFAR-100 with 20, 40, 60, or 80 classes
def select_classes_number(classes_number = 20):
  cifar100_20_classes_url = "https://pastebin.com/raw/nzE1n98V"
  cifar100_40_classes_url = "https://pastebin.com/raw/zGX4mCNP"
  cifar100_60_classes_url = "https://pastebin.com/raw/nsDTd3Qn"
  cifar100_80_classes_url = "https://pastebin.com/raw/SNbXz700"
  if classes_number == 20:
    return cifar100_20_classes_url
  elif classes_number == 40:
    return cifar100_40_classes_url
  elif classes_number == 60:
    return cifar100_60_classes_url
  elif classes_number == 80:
    return cifar100_80_classes_url
  else:
    return -1

In [28]:
import pandas as pd 

train_captions = pd.read_csv('/content/train_captions.csv', sep='\t',  header=None, quoting=3, error_bad_lines=False)
test_image = pd.read_csv('/content/test_images.csv', sep='\t',  header=None, quoting=3, error_bad_lines=False)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [32]:
train_captions.head()
test_image.head()

,0
0,_1000400860.jpg
1,_100046208.jpg
2,_1000933460.jpg
3,_1003969002.jpg
4,_10042424.jpg


In [34]:
caption_feattures = train_captions.copy()
caption_feattures = np.array(caption_feattures)
caption_feattures

array([['_1000070808.jpg| 0| A little girl holds orange lilies while others celebrate an occasion in a garden setting .'],
       ['_1000070808.jpg| 1| A little girl at a wedding holds a bouquet of orange flowers .'],
       ['_1000070808.jpg| 2| Young girl in orange dress holding orange flowers .'],
       ...,
       ['_99999606.jpg| 2| Older man with shoes in front and behind him .'],
       ['_99999606.jpg| 3| This man has worked here for many decades .'],
       ['_99999606.jpg| 4| an old shoemaker in his factory .']],
      dtype=object)

In [36]:
images_feattures = test_image.copy()
images_feattures = np.array(images_feattures)
images_feattures

array([['_1000400860.jpg'],
       ['_100046208.jpg'],
       ['_1000933460.jpg'],
       ...,
       ['_997103085.jpg'],
       ['_99794568.jpg'],
       ['_99922185.jpg']], dtype=object)

In [41]:
model = tf.keras.applications.nasnet.NASNetLarge(
    input_shape=None, include_top=True, weights='imagenet',
    input_tensor=None, pooling=None, classes=1000
)
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.0001), loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=["accuracy"])


In [44]:
def image_to_input(model, img_path):
    target_size=model.input_shape[1:]
    img = keras_preprocessing_image.load_img(img_path, target_size=target_size)
    
    x = keras_preprocessing_image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    return x

In [45]:
def get_single_prediction(img_path, top=5):
    x = image_to_input(model_imagenet, img_path)
    preds = model_imagenet.predict(x)
    predictions = decode_predictions(preds, top=top)
    return predictions[0]